# From images to coordinates

### Initial imports

In [1]:
import numpy as np
import cv2
from tqdm import tqdm
from glob import glob
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Input, Conv2D, Flatten, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
import keras.backend as K

### Loading images

In [2]:
# Dataset dependant parameters
data_path = "data/3ball/"
num_balls = 3
num_systems = 10000
color = True
pix = 32

X1 = []
for i in tqdm(glob(data_path + "Input/image1_*.png")):
    if color:
        im = cv2.imread(i)
    else:
        im = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    im = keras.utils.img_to_array(im)
    X1.append(im)
X1 = np.array(X1, dtype="float32") / 255

X2 = []
for i in tqdm(glob(data_path + "Input/image2_*.png")):
    if color:
        im = cv2.imread(i)
    else:
        im = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    im = keras.utils.img_to_array(im)
    X2.append(im)
X2 = np.array(X2, dtype="float32") / 255

100%|██████████| 10000/10000 [00:03<00:00, 2619.58it/s]


### Coordinates generation and data split

In [3]:
coordinates = np.loadtxt(data_path + "coordinates.csv", delimiter=",")
# Store the position on the second frame and velocity vectors bounded to 1 (and -1 or 0 depending on if takes negative values)
coordinates = np.concatenate(
    (
        coordinates[:, 2 * num_balls : 4 * num_balls] / (pix - 1),
        coordinates[:, -2 * num_balls :] / 4,
    ),
    axis=1,
)

# Concatenate both frames
X = np.concatenate((X1, X2), axis=3)
# Train-val-test split (80-10-10)
x_train, x_test, coordinates_train, coordinates_test = train_test_split(
    X, coordinates, test_size=0.2
)
x_val, x_test, coordinates_val, coordinates_test = train_test_split(
    x_test, coordinates_test, test_size=0.5
)

### Model definition and training

In [4]:
# Input layer
inputs = Input(shape=(pix, pix, 2 + 4 * color))

# Hidden layers
x = Conv2D(1, (3, 3), activation="relu")(inputs)
x = Conv2D(1, (3, 3), activation="relu")(x)
x = Conv2D(6, (28, 28), activation="relu")(x)
x = Flatten()(x)
x = Dense(12 * num_balls)(x)

# Output layer
outputs = Dense(4 * num_balls)(x)

model = Model(inputs, outputs, name="ImgToCoordinates")
model.summary()

# Free up RAM in case the model definition cells were run multiple times
K.clear_session()

Model: "ImgToCoordinates"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 6)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 1)         55        
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 1)         10        
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 6)           4710      
                                                                 
 flatten (Flatten)           (None, 6)                 0         
                                                                 
 dense (Dense)               (None, 36)                252       
                                                                 
 dense_1 (Dense)             (None, 12)           

In [5]:
model.compile(loss="mae", optimizer="adam")

model.fit(
    x_train,
    coordinates_train,
    epochs=100,
    batch_size=16,
    shuffle=True,
    validation_data=(x_val, coordinates_val),
    callbacks=[
        EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True, min_delta=0.0001
        )
    ],
)

Epoch 1/100
500/500 [==============================] - 11s 17ms/step - loss: 0.2408 - val_loss: 0.2244
Epoch 2/100
500/500 [==============================] - 5s 10ms/step - loss: 0.2218 - val_loss: 0.2210
Epoch 3/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2204 - val_loss: 0.2203
Epoch 4/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2185 - val_loss: 0.2169
Epoch 5/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2157 - val_loss: 0.2147
Epoch 6/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2139 - val_loss: 0.2117
Epoch 7/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2104 - val_loss: 0.2062
Epoch 8/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2054 - val_loss: 0.2047
Epoch 9/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2042 - val_loss: 0.2044
Epoch 10/100
500/500 [==============================] - 3s 6ms/step - loss: 0.2036 - val

### Model results

In [6]:
predictions = model.predict(x_test)

for i in range(5):
    print(
        str(i)
        + "\nPrediction: "
        + str(predictions[i])
        + "\nActual coordinates: "
        + str(coordinates_test[i])
    )

32/32 [==============================] - 0s 6ms/step
0
Prediction: [ 4.7458196e-01  3.4244683e-01  4.5479989e-01  3.3794156e-01
  4.5714450e-01  3.2935190e-01 -3.9898101e-03 -1.6008335e-01
  4.5656506e-04 -1.5499625e-01 -1.6034730e-03 -1.4070415e-01]
Actual coordinates: [ 0.4516129   0.61290323  0.22580645  0.22580645  0.80645161  0.29032258
  0.         -0.5        -0.5        -0.25        0.25        0.5       ]
1
Prediction: [4.2474294e-01 5.8809561e-01 4.1419682e-01 5.8947599e-01 3.9806744e-01
 5.8337849e-01 3.4215380e-03 1.1874834e-01 3.8462207e-03 1.3342339e-01
 5.1869079e-04 1.4195505e-01]
Actual coordinates: [ 0.4516129   0.74193548  0.32258065  0.58064516  0.64516129  0.22580645
  0.          0.25        0.          0.          0.         -0.25      ]
2
Prediction: [ 4.8165262e-01  4.8015001e-01  4.6362936e-01  4.7157559e-01
  4.5890650e-01  4.6778053e-01 -2.3308038e-03 -2.1976262e-01
  1.0068873e-03 -2.0865187e-01 -3.5719760e-04 -2.1667551e-01]
Actual coordinates: [ 0.7419354